In [1]:
import pandas as pd

In [3]:
tabla = pd.read_csv("tabla_pld.txt", sep="|")

In [5]:
tabla.head()

,Unnamed: 0,CD_CLIENTE,CD_DICTAMEN,CD_TIPOLOGIA,SEG_MOD,DES_RIE_TIP,DES_COT_BMV,DES_ID_SEC,DES_ID_SEG,DES_ID_EDO,...,SPE_NU_CTE_LI0,SPE_NU_CTE_LI1,SPI_NU_MES0,SPI_NU_MES1,SPI_AVG_MON0,SPI_AVG_MON1,SPI_NU_CTA0,SPI_NU_CTA1,CCC_NU_CTE_LI,Unnamed: 165
0,NaN,1,0,M2,3,0.5005,0,1,16,1,...,0,0,0,0,0.0,0.0,0,0,0,NaN
1,NaN,2,0,MD,2,0.4037,0,1,7,30,...,0,2,0,0,0.0,0.0,0,0,6,NaN
2,NaN,3,0,MP,4,0.8549,0,2,9,32,...,0,0,0,0,0.0,0.0,0,0,18,NaN
3,NaN,4,0,MP,4,0.8549,0,1,7,21,...,0,0,0,0,0.0,0.0,0,0,0,NaN
4,NaN,5,0,MD,2,0.4037,0,3,9,3,...,0,0,0,0,0.0,0.0,0,0,0,NaN


In [9]:
import pandas as pd
import numpy as np

from pyspark.sql import SQLContext, SparkSession, Row, DataFrame
from pyspark.sql.functions import when, count, col, isnan, isnull, mean, lit, regexp_replace
from pyspark.sql.types import StringType, DoubleType, BooleanType, DateType, IntegerType, StructType, StructField, LongType,ByteType, ArrayType
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import StringIndexer, QuantileDiscretizer, ChiSqSelector, OneHotEncoder, VectorAssembler, Imputer, StandardScaler, StringIndexer, OneHotEncoderEstimator, NGram, HashingTF, MinHashLSH, RegexTokenizer
from pyspark.ml.classification import LogisticRegression, LinearSVC, MultilayerPerceptronClassifier, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.stat import Correlation, ChiSquareTest
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from functools import reduce
import itertools

In [8]:
#!pip install -q findspark
import findspark
findspark.init()

In [10]:
# dir(SparkSession.builder)
spark = SparkSession.builder.config("spark.driver.memory", "10G").appName("ClasificacionBasemodelo").getOrCreate()

In [11]:
spark.sparkContext.getConf().getAll()

[('spark.driver.host', '192.168.0.12'),
 ('spark.driver.port', '56612'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.name', 'ClasificacionBasemodelo'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.memory', '10G'),
 ('spark.app.id', 'local-1591223729459'),
 ('spark.ui.showConsoleProgress', 'true')]

In [12]:
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l2
from sklearn import datasets
from tensorflow.keras import layers
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [ ]:
schema = StructType([
    StructField("DES_RIE_TIP", DoubleType()),
    StructField("DES_EDA", DoubleType()),
    StructField("DES_ANT_CTA", DoubleType()),
    StructField("DES_MON_DEC", IntegerType()),
    StructField("EFE_NU_MES_MOND", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
    StructField("EFE_SD_MON0", DoubleType()),
])

In [38]:
df = spark.read.option("delimiter", "|").option("header", True).csv("./tabla_pld.txt")

In [39]:
df.show()

+----+----------+-----------+------------+-------+-----------+-----------+----------+----------+----------+----------+---------+-------+-----------+-----------+-----------+---------------+---------------+---------------+--------------+---------------+--------------+---------------+---------------+--------------+---------------+---------------+---------------+---------------+--------------+-------------+-------------+---------------+---------------+-------------+-------------+-----------+-----------+------------+------------+---------------+---------------+----------------+----------------+---------------+---------------+----------------+---------------+------------+------------+------------+------------+------------+------------+---------------+------------+---------------+---------------+------------+------------+---------------+---------------+------------+------------+---------------+---------------+------------+------------+------------+------------+---------------+---------------+

In [40]:
df.dtypes

[('_c0', 'string'),
 ('CD_CLIENTE', 'string'),
 ('CD_DICTAMEN', 'string'),
 ('CD_TIPOLOGIA', 'string'),
 ('SEG_MOD', 'string'),
 ('DES_RIE_TIP', 'string'),
 ('DES_COT_BMV', 'string'),
 ('DES_ID_SEC', 'string'),
 ('DES_ID_SEG', 'string'),
 ('DES_ID_EDO', 'string'),
 ('DES_ID_BAN', 'string'),
 ('DES_ID_RB', 'string'),
 ('DES_EDA', 'string'),
 ('DES_ANT_CTE', 'string'),
 ('DES_ANT_CTA', 'string'),
 ('DES_MON_DEC', 'string'),
 ('EFE_NU_MES_MON0', 'string'),
 ('EFE_NU_MES_MON1', 'string'),
 ('EFE_NU_MES_MOND', 'string'),
 ('   EFE_SD_MON0', 'string'),
 ('    EFE_SD_MON1', 'string'),
 ('   EFE_SD_MOND', 'string'),
 ('EFE_NU_MES_MOV0', 'string'),
 ('EFE_NU_MES_MOV1', 'string'),
 ('   EFE_SD_MOV0', 'string'),
 ('    EFE_SD_MOV1', 'string'),
 ('OPI_NU_MES_MON0', 'string'),
 ('OPI_NU_MES_MON1', 'string'),
 ('OPI_NU_MES_MOND', 'string'),
 ('   OPI_SD_MON0', 'string'),
 ('  OPI_SD_MON1', 'string'),
 ('  OPI_SD_MOND', 'string'),
 ('OPI_NU_MES_MOV0', 'string'),
 ('OPI_NU_MES_MOV1', 'string'),
 ('  O

In [41]:
df.select("DES_COT_BMV").distinct().collect()

[Row(DES_COT_BMV='          0')]

In [47]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- CD_CLIENTE: string (nullable = true)
 |-- CD_DICTAMEN: string (nullable = true)
 |-- CD_TIPOLOGIA: string (nullable = true)
 |-- SEG_MOD: string (nullable = true)
 |-- DES_RIE_TIP: double (nullable = true)
 |-- DES_COT_BMV: string (nullable = true)
 |-- DES_ID_SEC: string (nullable = true)
 |-- DES_ID_SEG: string (nullable = true)
 |-- DES_ID_EDO: string (nullable = true)
 |-- DES_ID_BAN: string (nullable = true)
 |-- DES_ID_RB: string (nullable = true)
 |-- DES_EDA: double (nullable = true)
 |-- DES_ANT_CTE: string (nullable = true)
 |-- DES_ANT_CTA: string (nullable = true)
 |-- DES_MON_DEC: string (nullable = true)
 |-- EFE_NU_MES_MON0: string (nullable = true)
 |-- EFE_NU_MES_MON1: string (nullable = true)
 |-- EFE_NU_MES_MOND: string (nullable = true)
 |--    EFE_SD_MON0: string (nullable = true)
 |--     EFE_SD_MON1: string (nullable = true)
 |--    EFE_SD_MOND: string (nullable = true)
 |-- EFE_NU_MES_MOV0: string (nullable = true)
 |